In [ ]:
from hydromt_sfincs import SfincsModel
from os.path import join, isfile, basename, isdir, dirname
import xarray as xr
import numpy as np
import glob
import hydromt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from string import ascii_uppercase as abcd


In [ ]:
model_root = r'../../3_models/SFINCS'
fdir = r'../../4_results'
base_root = join(model_root, "01_rivpowlaw")
events = {'idai': ('20190314', '20190322'), 'eloise':('20210119', '20210128')}


In [ ]:
df_obs = pd.read_csv(join(r'../../1_data/4_observations', 'hydrological_bulletin_waterlevels.csv'), index_col=0, parse_dates=True)
df_obs.columns = df_obs.columns.astype(int)

In [ ]:
sfx = {}
for j, (event, dates) in enumerate(events.items()):
    mod = SfincsModel(join(base_root, event), mode='r')
    da_sfx_zs = mod.results['point_zs'].sel(time=slice(*dates)).rename({'stations': 'index'})
    da_sfx_zs.vector.set_spatial_dims(x_dim='station_x', y_dim='station_y', index_dim='index')
    da_sfx_zs['index'] = da_sfx_zs['index']+1
    sfx[event] = da_sfx_zs.load()
da_sfx_zs.vector.to_gdf()

In [ ]:
# read CMF waterlevels for combined idai + eloise runs
root = r'../../3_models/CMF/03min'
# linear indices at 3min res
# 1-5 correspond to SFINCS gauge locations
# 6-7 correspond to approx observed locations
idxs = np.array([4108, 4112, 3410,3550,3904, 3959, 3480])
ncol=70
cols = xr.IndexVariable('index', idxs%ncol)
rows = xr.IndexVariable('index', idxs//ncol)
# name = 'flddph'#
name = 'rivdph'

cmf_out = join(root, '01_powlaw')
da_cmf_z0 = hydromt.open_raster(join(root, 'elevtn.tif'))
da_cmf_h = xr.open_mfdataset(join(cmf_out, f'o_{name}20*.nc'))[name].rename('zs')
if name == 'flddph':
    da_cmf_zs = da_cmf_h + da_cmf_z0.raster.reproject_like(da_cmf_h)
else:
    da_cmf_zb = da_cmf_z0 - hydromt.open_raster(join(root, 'rivdph_hc0.27_hp0.3.tif'))
    da_cmf_zs = da_cmf_h + da_cmf_zb.raster.reproject_like(da_cmf_h)

cmf =  {}
for j, (event, dates) in enumerate(events.items()):
    da_cmf_zs0 = da_cmf_zs.isel(lon=cols.isel(index=slice(0,5)), lat=rows.isel(index=slice(0,5))).sel(time=slice(*dates))
    da_cmf_zs0['index'] = da_cmf_zs0['index']+1
    cmf[event] = da_cmf_zs0.load()

da_cmf_zs0.vector.to_gdf()

In [ ]:
# plot SFINCS vs CaMa-Flood simulated water levels

n = 2
kwargs0 = dict(figsize=(16, 8))
fig, axes = plt.subplots(2,2, **kwargs0)
colors = ['darkblue', 'darkgreen', 'lightblue', 'darkmagenta', 'orangered']

# df.index = mdates.date2num(df.index)
for icol, (event, dates) in enumerate(events.items()):
    locator = mdates.AutoDateLocator()
    formatter = mdates.ConciseDateFormatter(locator)

    ax = axes[0,icol]
    j = 0
    for i in range(1,3):
        sfx[event].sel(index=i).plot.line(ax=ax, x='time', color=colors[j], label=f'{i} - SFINCS')
        cmf[event].sel(index=i).plot.line(ax=ax, x='time', color=colors[j], ls='--', label=f'{i} - CaMa-Flood')
        j += 1
    ax.set_xlim([sfx[event].time.values[0], sfx[event].time.values[-1]])
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    ax.set_xticklabels('')
    ax.set_xlabel('')
    if icol == 1:
        ax.set_ylim(axes[0,0].get_ylim())
        ax.set_yticklabels('')
        ax.legend(
            title='observation points Buzi River',
            bbox_to_anchor=(1, 1.02),
            loc="upper right",
            ncol=2,
        )
    else:
        ymin, ymax = ax.get_ylim()
        ax.set_ylim([ymin, ymax*1.1])
        ax.set_ylabel('waterlevel [m+EGM96]')
    ax.set_title(event.capitalize())
    ax.text(0.01, 0.9, abcd[icol].upper(), fontsize=14, fontweight='bold', transform=ax.transAxes)


    ax1 = axes[1,icol]

    for i in range(3,6):
        sfx[event].sel(index=i).plot.line(ax=ax1, x='time', color=colors[j], label=f'{i} - SFINCS')
        cmf[event].sel(index=i).plot.line(ax=ax1, x='time', color=colors[j], ls='--', label=f'{i} - CaMa-Flood')
        j += 1
    if icol == 1:
        ax1.set_ylim(axes[1,0].get_ylim())
        ax1.set_yticklabels('')
        ax1.legend(
            title='observation points Pungwe River',
            bbox_to_anchor=(1, 1.1),
            loc="upper right",
            ncol=3,
        )
    else:
        ymin, ymax = ax1.get_ylim()
        ax1.set_ylim([ymin, ymax*1.1])
        ax1.set_ylabel('waterlevel [m+EGM96]')
    ax1.set_title('')
    ax1.set_xlim([sfx[event].time.values[0], sfx[event].time.values[-1]])
    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    ax1.text(0.01, 0.9, abcd[icol+1].upper(), fontsize=14, fontweight='bold', transform=ax1.transAxes)


fig.subplots_adjust(wspace=0.05, hspace=0.05) 

plt.savefig(join(fdir, f'sfincs_vs_cmf{name}.png'), dpi=225, bbox_inches="tight")

In [ ]:
da_cmf_zs.isel(lon=cols.isel(index=slice(5,7)), lat=rows.isel(index=slice(5,7))).to_series().unstack()

In [ ]:
# plot CaMa-Flood vs observed water levels

import matplotlib.pyplot as plt

df = da_cmf_h.isel(lon=cols.isel(index=slice(5,7)), lat=rows.isel(index=slice(5,7))).to_series().unstack()
df.columns = df.columns + 1

fig, (ax, ax1) = plt.subplots(2,1, figsize = (8,6), sharex=True)
axb = ax.twinx()
df[1].plot(ax=ax, label='sim - CaMa-Flood', color='tab:blue')
df_obs[1].plot(ax=axb, lw=0, marker='o', label='obs', color='k')
axb.set_ylabel('obs waterlevel [m+REF]')
ax.set_ylabel('sim water depth [m]')
ax.legend(loc='upper left')
axb.legend(loc='upper right')
ax.set_title('Buzi river @ Goonda')

ax1b = ax1.twinx()
df[2].plot(ax=ax1, label='sim - CaMa-Flood', color='tab:blue')
# df1[2].plot(ax=ax1, label='sim - GLOFAS', color='orangered', ls='--')
df_obs[2].plot(ax=ax1b, lw=0, marker='o', label='obs', color='k')
ax1b.set_ylabel('obs waterlevel [m+REF]')
ax1.set_ylabel('sim water depth [m]')
ax1.legend(loc='upper left')
ax1b.legend(loc='upper right')
ax1.set_title('Pungwe river @ Mafambisse')

# ax1.set_xticklabels('')
# locator = mdates.AutoDateLocator()
# formatter = mdates.ConciseDateFormatter(locator)
# ax1b.xaxis.set_major_locator(locator)
# ax1b.xaxis.set_major_formatter(formatter)
plt.xticks(rotation=0)

plt.savefig(join(fdir, 'obs_waterlevels.png'), dpi=300,)